In [2]:
import os
from intuitlib.client import AuthClient
from quickbooks import QuickBooks
from quickbooks.objects.account import Account
import pandas as pd
import numpy as np
import calendar
from dateutil import parser
import yaml
import sqlite3

In [3]:
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.getcwd()))

In [4]:
def get_auth_client(client_id, client_secret, refresh_token, company_id):
    auth_client = AuthClient(
            client_id=client_id,
            client_secret=client_secret,
            access_token=None,
            environment='production',
            redirect_uri='https://developer.intuit.com/v2/OAuth2Playground/RedirectUrl',
        )
    client = QuickBooks(
            auth_client=auth_client,
            refresh_token=refresh_token,
            company_id=company_id,
        )
    return client

In [5]:
def proc_rows(rows:list, category:str = "", level:int=0):
    row_list = []
    for row in rows:
        if "Header" in row:
            header_col = row['Header']['ColData'][0]['value']
            if category == "":
                current_category = header_col
            else:
                current_category = f"{category}:{header_col}"
            row_list.extend(proc_rows(row['Rows']['Row'], category = current_category, level = level+1))
        else:
            col_data = row['ColData']
            if len(col_data)==len(cols):
                cur_row = {cols[i]:col_data[i]['value'] for i in range(len(cols))}
                cur_row.update({"category":category})
                cur_row.update({"category_level":level})
                row_list.append(cur_row)
    return row_list

In [6]:
def load_yaml(yaml_file:str):
    with open(yaml_file, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
            sys.exit(1)

In [7]:
credentials = load_yaml(os.path.join(SCRIPT_DIR,"config","credential.yaml"))

In [8]:
year = 2023
client = get_auth_client(credentials['client_id'],credentials['client_secret'],
                         credentials['refresh_token'],credentials['company_id'])
df_list = []
for month in range(1,13):
    days = calendar.monthrange(year,month)[1]
    month_name = calendar.month_name[month]
    print(f"Grabbing report details for {month_name}")
    json_resp = client.get_report("ProfitAndLossDetail", {"start_date":f"{year}-{month}-01", "end_date":f"{year}-{month}-{days}"})
    cols = [i["ColTitle"] for i in json_resp['Columns']['Column']]
    report_info = json_resp['Header']
    if "Row" not in json_resp["Rows"]:
        print("No data for this month...skipping")
        continue
    row_list = proc_rows(json_resp["Rows"]["Row"][0]['Rows']['Row'])
    df_list.append(pd.DataFrame(row_list))

Grabbing report details for January
Grabbing report details for February
Grabbing report details for March
Grabbing report details for April
Grabbing report details for May
Grabbing report details for June
Grabbing report details for July
Grabbing report details for August
Grabbing report details for September
Grabbing report details for October
No data for this month...skipping
Grabbing report details for November
No data for this month...skipping
Grabbing report details for December
No data for this month...skipping


In [10]:
qbdf = pd.concat(df_list)
conn = sqlite3.connect("quickbooks_db")
qbdf.to_sql('categorized_items', conn, if_exists='replace', index=False)

392

In [12]:
df = pd.read_sql("select * from categorized_items", conn)

In [13]:
df

,Date,Transaction Type,Num,Name,Memo/Description,Split,Amount,Balance,category,category_level
0,2023-01-31,Deposit,,,"Deposit Dividend 0.050% APY Earn Dividend, Dep...",ENT Savings (Keystone),.40,.40,Income:Interest Earned,2
1,2023-01-03,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),145.35,145.35,Income:Tithe:General Tithe,3
2,2023-01-08,Deposit,,Larry Metzler,Metzler direct deposit,1002 ENT Checking (Keystone),1280.00,1425.35,Income:Tithe:General Tithe,3
3,2023-01-09,Deposit,,,"Deposit Bulk Checks 1,30 Deposit, Proc...",1002 ENT Checking (Keystone),1300.00,2725.35,Income:Tithe:General Tithe,3
4,2023-01-09,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),155.00,2880.35,Income:Tithe:General Tithe,3
...,...,...,...,...,...,...,...,...,...,...
387,2023-09-01,Expense,,Code 3 Lawn Rescue,"Visa Check SQ *CODE 3 LAWN RESC DebitCard, Wi...",1002 ENT Checking (Keystone),75.00,75.00,Expenses:Facilities:Government/Professional Fe...,4
388,2023-09-01,Expense,,Amazon,"Visa Check AMZN Mktp US*TL9DO9L DebitCard, Wi...",1002 ENT Checking (Keystone),46.69,46.69,Expenses:Facilities:Hospitality:Food for Event...,4
389,2023-09-01,Expense,,Google,"Visa Check GOOGLE GSUITE_mykeys DebitCard, Wi...",1002 ENT Checking (Keystone),9.28,9.28,Expenses:Facilities:Software and Subscriptions,3
390,2023-09-01,Expense,,Mission Coffee Roast,"Visa Check SQ *THIRD SPACE COFF DebitCard, Wi...",1002 ENT Checking (Keystone),25.52,25.52,Expenses:Ministry:Individual Ministries:Coffee...,4
